Importing the libraries

In [1]:
import requests
from collections import Counter
from bs4 import BeautifulSoup
import pandas as pd
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
from textblob import TextBlob
from wordcloud import WordCloud
from string import digits
import requests
import pandas as pd
import nltk
import string
import seaborn as sns
import re
nltk.download("stopwords")
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Web scraping the reviews from the website "https://www.airlinequality.com/airline-reviews/british-airways"

In [2]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"
    print(url)
    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())

    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
https://www.airlinequality.com/airline-reviews/british-airways/page/1/?sortby=post_date%3ADesc&pagesize=100
   ---> 100 total reviews
Scraping page 2
https://www.airlinequality.com/airline-reviews/british-airways/page/2/?sortby=post_date%3ADesc&pagesize=100
   ---> 200 total reviews
Scraping page 3
https://www.airlinequality.com/airline-reviews/british-airways/page/3/?sortby=post_date%3ADesc&pagesize=100
   ---> 300 total reviews
Scraping page 4
https://www.airlinequality.com/airline-reviews/british-airways/page/4/?sortby=post_date%3ADesc&pagesize=100
   ---> 400 total reviews
Scraping page 5
https://www.airlinequality.com/airline-reviews/british-airways/page/5/?sortby=post_date%3ADesc&pagesize=100
   ---> 500 total reviews
Scraping page 6
https://www.airlinequality.com/airline-reviews/british-airways/page/6/?sortby=post_date%3ADesc&pagesize=100
   ---> 600 total reviews
Scraping page 7
https://www.airlinequality.com/airline-reviews/british-airways/page/7/?sortby=post_d

Loading data

In [30]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()
df.to_csv("data/ReviewsData.csv")

**Data Cleaning**

In [31]:
df['verification_status'] = df['reviews'].str.extract('(✅ Trip Verified|Not Verified)', expand=False)
df['cleaned_reviews'] = df['reviews'].str.replace('(✅ Trip Verified|Not Verified)', '').str.strip().str.lower()


<ipython-input-31-dec38f783d73>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['cleaned_reviews'] = df['reviews'].str.replace('(✅ Trip Verified|Not Verified)', '').str.strip().str.lower()


In [32]:
df.drop(columns=['reviews'], inplace=True)

In [40]:
print(df)

    verification_status                                    cleaned_reviews
0         Trip Verified  never i saw such a terrible customer experienc...
1         Trip Verified  having not flown with ba for a few years, i re...
2         Trip Verified  dear community i feel compelled to share the u...
3         Trip Verified  quick bag drop at first wing but too many pass...
4         Trip Verified  4 hours before takeoff we received a mail stat...
..                  ...                                                ...
995       Trip Verified  antigua to gatwick. apparently it is necessary...
996       Trip Verified  i understand completely why there is air rage ...
997       Trip Verified  london to miami. worst long haul business clas...
998       Trip Verified  i used avios point to upgrade from economy to ...
999       Trip Verified  boarding was fairly quick, the well turned out...

[1000 rows x 2 columns]


In [34]:
df['cleaned_reviews'] = df['cleaned_reviews'].str.split('|').str[-1].str.strip()


In [35]:
df['verification_status'] = df['verification_status'].str.replace('✅ ', '').str.strip()

print(df)

    verification_status                                    cleaned_reviews
0         Trip Verified  never i saw such a terrible customer experienc...
1         Trip Verified  having not flown with ba for a few years, i re...
2         Trip Verified  dear community i feel compelled to share the u...
3         Trip Verified  quick bag drop at first wing but too many pass...
4         Trip Verified  4 hours before takeoff we received a mail stat...
..                  ...                                                ...
995       Trip Verified  antigua to gatwick. apparently it is necessary...
996       Trip Verified  i understand completely why there is air rage ...
997       Trip Verified  london to miami. worst long haul business clas...
998       Trip Verified  i used avios point to upgrade from economy to ...
999       Trip Verified  boarding was fairly quick, the well turned out...

[1000 rows x 2 columns]


In [38]:
distinct_values = df['verification_status'].unique()

# Displaying the distinct values
print(distinct_values)

['Trip Verified' 'Not Verified']


In [39]:
value_counts = df['verification_status'].value_counts()
value_counts_df = value_counts.reset_index()
value_counts_df.columns = ['Value', 'Count']

# Displaying the DataFrame
print(value_counts_df)

           Value  Count
0  Trip Verified    780
1   Not Verified    220


In [41]:
df.rename(columns={'cleaned_reviews': 'reviews'}, inplace=True)

In [42]:
df['reviews'] = df['reviews'].str.replace('[{}]'.format(string.punctuation), '')
print(df)

    verification_status                                            reviews
0         Trip Verified  never i saw such a terrible customer experienc...
1         Trip Verified  having not flown with ba for a few years i rea...
2         Trip Verified  dear community i feel compelled to share the u...
3         Trip Verified  quick bag drop at first wing but too many pass...
4         Trip Verified  4 hours before takeoff we received a mail stat...
..                  ...                                                ...
995       Trip Verified  antigua to gatwick apparently it is necessary ...
996       Trip Verified  i understand completely why there is air rage ...
997       Trip Verified  london to miami worst long haul business class...
998       Trip Verified  i used avios point to upgrade from economy to ...
999       Trip Verified  boarding was fairly quick the well turned out ...

[1000 rows x 2 columns]


<ipython-input-42-ef40ba0536d5>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['reviews'] = df['reviews'].str.replace('[{}]'.format(string.punctuation), '')


In [45]:
nltk.download('punkt')
df['tokenized_review'] = df['reviews'].apply(lambda x: word_tokenize(x))

print(df)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


    verification_status                                            reviews  \
0         Trip Verified  never i saw such a terrible customer experienc...   
1         Trip Verified  having not flown with ba for a few years i rea...   
2         Trip Verified  dear community i feel compelled to share the u...   
3         Trip Verified  quick bag drop at first wing but too many pass...   
4         Trip Verified  4 hours before takeoff we received a mail stat...   
..                  ...                                                ...   
995       Trip Verified  antigua to gatwick apparently it is necessary ...   
996       Trip Verified  i understand completely why there is air rage ...   
997       Trip Verified  london to miami worst long haul business class...   
998       Trip Verified  i used avios point to upgrade from economy to ...   
999       Trip Verified  boarding was fairly quick the well turned out ...   

                                      tokenized_review  
0    [

In [52]:
df.rename(columns={'tokenized_review': 'reviews'}, inplace=True)
df.head(10)

,verification_status,reviews
0,Trip Verified,"[never, i, saw, such, a, terrible, customer, e..."
1,Trip Verified,"[having, not, flown, with, ba, for, a, few, ye..."
2,Trip Verified,"[dear, community, i, feel, compelled, to, shar..."
3,Trip Verified,"[quick, bag, drop, at, first, wing, but, too, ..."
4,Trip Verified,"[4, hours, before, takeoff, we, received, a, m..."
5,Trip Verified,"[i, recently, had, a, delay, on, british, airw..."
6,Not Verified,"[boarded, on, time, but, it, took, ages, to, g..."
7,Trip Verified,"[5, days, before, the, flight, we, were, advis..."
8,Not Verified,"[we, traveled, to, lisbon, for, our, dream, va..."
9,Trip Verified,"[booked, a, flight, from, bucharest, to, manch..."


In [47]:
print(df.iloc[0,1])

['never', 'i', 'saw', 'such', 'a', 'terrible', 'customer', 'experience', 'the', 'plane', 'landed', 'already', '30', 'minutes', 'late', 'despite', 'any', 'operational', 'delays', 'the', 'boarding', 'took', 'over', '1', 'hour', 'because', 'the', 'crew', 'did', 'not', 'know', 'where', 'to', 'place', 'the', 'luggages', 'after', 'this', 'long', '1', 'hour', 'boarding', 'process', 'the', 'captain', 'did', 'not', 'apologise', 'this', 'is', 'extremely', 'frustrating', 'when', 'this', 'occurs', 'on', 'a', 'night', 'flight', 'forcing', 'passengers', 'to', 'lose', 'train', 'connections', 'to', 'home']


In [53]:
df['filtered_review'] = df['reviews'].apply(lambda tokens: [word for word in tokens if word.lower() not in stopwords.words('english')])

print(df)

    verification_status                                            reviews  \
0         Trip Verified  [never, i, saw, such, a, terrible, customer, e...   
1         Trip Verified  [having, not, flown, with, ba, for, a, few, ye...   
2         Trip Verified  [dear, community, i, feel, compelled, to, shar...   
3         Trip Verified  [quick, bag, drop, at, first, wing, but, too, ...   
4         Trip Verified  [4, hours, before, takeoff, we, received, a, m...   
..                  ...                                                ...   
995       Trip Verified  [antigua, to, gatwick, apparently, it, is, nec...   
996       Trip Verified  [i, understand, completely, why, there, is, ai...   
997       Trip Verified  [london, to, miami, worst, long, haul, busines...   
998       Trip Verified  [i, used, avios, point, to, upgrade, from, eco...   
999       Trip Verified  [boarding, was, fairly, quick, the, well, turn...   

                                       filtered_review  
0    [

In [54]:
df.drop(columns=['reviews'], inplace=True)
df.rename(columns={'filtered_review': 'reviews'}, inplace=True)
df.head()

,verification_status,reviews
0,Trip Verified,"[never, saw, terrible, customer, experience, p..."
1,Trip Verified,"[flown, ba, years, read, reviews, preparation,..."
2,Trip Verified,"[dear, community, feel, compelled, share, utte..."
3,Trip Verified,"[quick, bag, drop, first, wing, many, passenge..."
4,Trip Verified,"[4, hours, takeoff, received, mail, stating, c..."


In [61]:
df['reviews'] = df['reviews'].astype(str).str.replace(']', '').str.replace(']', '')
df['reviews'] = df['reviews'].astype(str).str.replace('[', '').str.replace(']', '')
df.head()

<ipython-input-61-a848a96226a9>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['reviews'] = df['reviews'].astype(str).str.replace(']', '').str.replace(']', '')
<ipython-input-61-a848a96226a9>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['reviews'] = df['reviews'].astype(str).str.replace('[', '').str.replace(']', '')


,verification_status,reviews,new
0,Trip Verified,"'never', 'saw', 'terrible', 'customer', 'exper...","'never', 'saw', 'terrible', 'customer', 'exper..."
1,Trip Verified,"'flown', 'ba', 'years', 'read', 'reviews', 'pr...","'flown', 'ba', 'years', 'read', 'reviews', 'pr..."
2,Trip Verified,"'dear', 'community', 'feel', 'compelled', 'sha...","'dear', 'community', 'feel', 'compelled', 'sha..."
3,Trip Verified,"'quick', 'bag', 'drop', 'first', 'wing', 'many...","'quick', 'bag', 'drop', 'first', 'wing', 'many..."
4,Trip Verified,"'4', 'hours', 'takeoff', 'received', 'mail', '...","'4', 'hours', 'takeoff', 'received', 'mail', '..."


In [62]:
df.drop(columns=['new'], inplace=True)

In [72]:
def polarity_calc(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None



def tag_cal(num):
    if num<0:
        return 'Negative'
    elif num>0:
        return 'Positive'
    else:
        return 'Neutral'


df['polarity'] = df['reviews'].apply(polarity_calc)


df['tag'] = df['polarity'].apply(tag_cal)


print(df)

    verification_status                                            reviews  \
0         Trip Verified  'never', 'saw', 'terrible', 'customer', 'exper...   
1         Trip Verified  'flown', 'ba', 'years', 'read', 'reviews', 'pr...   
2         Trip Verified  'dear', 'community', 'feel', 'compelled', 'sha...   
3         Trip Verified  'quick', 'bag', 'drop', 'first', 'wing', 'many...   
4         Trip Verified  '4', 'hours', 'takeoff', 'received', 'mail', '...   
..                  ...                                                ...   
995       Trip Verified  'antigua', 'gatwick', 'apparently', 'necessary...   
996       Trip Verified  'understand', 'completely', 'air', 'rage', 'sk...   
997       Trip Verified  'london', 'miami', 'worst', 'long', 'haul', 'b...   
998       Trip Verified  'used', 'avios', 'point', 'upgrade', 'economy'...   
999       Trip Verified  'boarding', 'fairly', 'quick', 'well', 'turned...   

     polarity       tag  polarity1 sentiment_tag1  
0   -0.4375

In [66]:
(df.groupby('tag').size()/df['tag'].count())*100

tag
Negative    34.3
Neutral      1.6
Positive    64.1
dtype: float64

In [74]:
df.head()

,verification_status,reviews,polarity,tag
0,Trip Verified,"'never', 'saw', 'terrible', 'customer', 'exper...",-0.437500,Negative
1,Trip Verified,"'flown', 'ba', 'years', 'read', 'reviews', 'pr...",-0.092708,Negative
2,Trip Verified,"'dear', 'community', 'feel', 'compelled', 'sha...",-0.005411,Negative
3,Trip Verified,"'quick', 'bag', 'drop', 'first', 'wing', 'many...",0.420000,Positive
4,Trip Verified,"'4', 'hours', 'takeoff', 'received', 'mail', '...",-0.048214,Negative


In [75]:
df.to_csv('data/reviewTrans.csv', index=False)

In [76]:
from collections import Counter

# Assuming df1 is your DataFrame and 'reviews' is the column containing text
# Replace 'reviews' with your actual column name
all_reviews = ' '.join(df['reviews'].dropna())

# Tokenize the text
words = all_reviews.split()

# Count the occurrences of each word
word_counts = Counter(words)

# Find the most common word
df1 = pd.DataFrame(list(word_counts.items()), columns=['word', 'occurrence'])

# Sort the DataFrame by occurrence in descending order
df1 = df1.sort_values(by='occurrence', ascending=False)

# Displaying the DataFrame with the most occurring words
print(df1.head(20))

            word  occurrence
30     'flight',        1842
38         'ba',        1090
340   'service',         691
521         '’',         579
116    'london',         547
299      'time',         486
295      'seat',         445
294      'food',         443
18       'crew',         433
96    'british',         421
322       'one',         410
60       'good',         410
386     'class',         406
357        'us',         399
351     'staff',         399
369     'seats',         398
97    'airways',         391
961  'business',         374
229       'get',         369
356     'would',         367


In [81]:
df1['word'] = df1['word'].astype(str).str.replace(',', '').str.replace(']', '')

<ipython-input-81-22af45657797>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df1['word'] = df1['word'].astype(str).str.replace(',', '').str.replace(']', '')


In [83]:
df1['word'] = df1['word'].astype(str).str.replace("'", '').str.replace(']', '')
print(df1.head(20))

         word  occurrence
30     flight        1842
38         ba        1090
340   service         691
521         ’         579
116    london         547
299      time         486
295      seat         445
294      food         443
18       crew         433
96    british         421
322       one         410
60       good         410
386     class         406
357        us         399
351     staff         399
369     seats         398
97    airways         391
961  business         374
229       get         369
356     would         367


<ipython-input-83-d7e8fea29aa4>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df1['word'] = df1['word'].astype(str).str.replace("'", '').str.replace(']', '')


In [84]:
df1.to_csv('data/topWords.csv', index=False)

In [88]:
print(df1.word.head(10))

30      flight
38          ba
340    service
521          ’
116     london
299       time
295       seat
294       food
18        crew
96     british
Name: word, dtype: object
